In [1]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
## We are going to create a lookup table which contain features for our models
# We create two tables: one for the source + references and one for the hypotheses
# First table is indexed by the source id and we use it to get features for the source and references
# The second table is indexed with the hypothesis id and can be used to query features for the hypotheses
# This to dramatically speed up the process.


In [2]:
import sacrebleu

In [88]:
sacrebleu.sentence_chrf("this is a sentence", ["this is sentence"], beta=2, char_order=1, word_order=0 )

chrF2 = 98.59

In [89]:
from nltk import word_tokenize, ngrams
from collections import Counter

beta = 2

sum_p = 0
sum_r = 0
hyp = "this is a sentence".replace(' ', '')
ref = "this is sentence".replace(' ', '')
for n in range(1, 2):
    hyp_chr_counter = Counter(ngrams(hyp.replace(' ', ''), n=n))
    ref_chr_counter = Counter(ngrams(ref.replace(' ', ''), n=n))
    
    total_hyp = sum(hyp_chr_counter.values())
    total_ref = sum(ref_chr_counter.values())
    
    hyp_chr_set = set(hyp_chr_counter.keys())
    ref_chr_set = set(ref_chr_counter.keys())
    matches = list(hyp_chr_set.intersection(ref_chr_set))
    
    counterpart_count_in_hyp = 0
    counterpart_count_in_ref = 0
    for m in matches:
        counterpart_count_in_hyp += hyp_chr_counter[m]
        counterpart_count_in_ref += ref_chr_counter[m]
    

    
    len_matches = len(matches)
    sum_p += counterpart_count_in_hyp / total_hyp if total_hyp else 0.
    sum_r += counterpart_count_in_ref / total_ref if total_ref else 0.

print(hyp_chr_counter)
    
# for n in range(1, 3):
#     hyp_chr_set = set(ngrams(word_tokenize(hyp), n=n))
#     ref_chr_set = set(ngrams(word_tokenize(ref), n=n))
#     matches = hyp_chr_set.intersection(ref_chr_set)
#     len_matches = len(matches)
#     sum_p += len_matches / len(hyp_chr_set) if len(hyp_chr_set) else 0.
#     sum_r += len_matches / len(ref_chr_set) if len(ref_chr_set) else 0.

    
avg_p = sum_p / 1
avg_r = sum_r / 1

s = 0. if (avg_p + avg_r) == 0. else (1 + beta**2) * ( avg_p * avg_r) / ((beta**2) * avg_p + avg_r)
print(s)

# print(s)

Counter({('s',): 3, ('e',): 3, ('t',): 2, ('i',): 2, ('n',): 2, ('h',): 1, ('a',): 1, ('c',): 1})
0.9859154929577465


In [16]:

matches = hyp_chr_set.intersection(ref_chr_set)
n = len(matches)
p = n / len(hyp_chr_set) if len(hyp_chr_set) else 0.
r = n / len(ref_chr_set) if len(ref_chr_set) else 0.
s = 0. if (p + r) == 0. else 2. * p * r / (p + r)


In [ ]:
from nltk import word_tokenize, ngrams

def fast_character_f( hyps, references):
    
    for hyp in hyps:
        for ref in references:
            ref_chr_ngrams = ngrams(ref)
        hyp_chr_ngrams = ngrams(hyp)
    

In [4]:
## We start with contructing the hypothesis table

df = pd.read_parquet('../data/comet/ancestral_10_100_train_predictive_0_smoke_test.parquet')


In [4]:

def create_hyp_ids(x):
    idx = x.name
    
    hyp_idx = []
    for i in range(len(x["hypotheses"])):
        hyp_idx.append('{}_{}'.format(idx, i))
    return hyp_idx

df["hypotheses_idx"] = df.apply(create_hyp_ids, axis=1)

In [5]:
df

,source,target,hypotheses,references_count,utilities,hypotheses_idx
0,Ich habe dich nicht hereinkommen gehört.,I didn't hear you come in.,"[I didn't hear you come in., I didn't hear you...","[9, 75, 3, 4, 1, 1, 2, 3, 1, 1]","[[0.9034017, 1.0596288, 0.80704784, 0.8429661,...","[0_0, 0_1]"
1,Du wirst Tom küssen müssen.,You'll have to kiss Tom.,"[You're going to have to kiss Tom., You're goi...","[75, 21, 4]","[[1.1020814, 0.89811295, 0.96650434], [1.00817...","[1_0, 1_1, 1_2]"
2,"Je mehr der Teufel hat, umso mehr will er.","The more the devil has, the more he wants.","[The more the devil has, the more he wants., T...","[67, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 4, 2, 1,...","[[1.0521425, 0.79392344, 0.3597772, -0.1302790...","[2_0, 2_1, 2_2, 2_3]"
3,Mich hat das auch nicht überrascht.,"That didn't surprise me, either.","[I wasn't surprised at it either., That doesn'...","[2, 1, 1, 4, 1, 1, 4, 1, 1, 1, 1, 1, 1, 6, 5, ...","[[1.1895545, 0.8660941, 0.94124323, 0.941016, ...","[3_0, 3_1, 3_2, 3_3, 3_4, 3_5, 3_6, 3_7, 3_8]"
4,Jedes Problem kann gelöst werden.,Every problem can be solved.,"[The problem can be solved., All of the proble...","[1, 30, 1, 24, 5, 7, 1, 1, 1, 3, 2, 1, 1, 2, 1...","[[0.6497966, 0.91674376, 1.0365593, 0.94450873...","[4_0, 4_1, 4_2, 4_3]"
...,...,...,...,...,...,...
96,Wie lang ist es?,How long is it?,[How long is it?],"[96, 1, 1, 1, 1]","[[1.1942372, 0.37009794, 0.82760555, -0.202470...",[96_0]
97,Wir haben kein Bier mehr.,We're out of beer.,"[We've run out of beer., We're out of beer., W...","[50, 17, 3, 5, 17, 2, 2, 1, 1, 1, 1]","[[1.1632272, -0.17549993, -0.18429604, -0.3144...","[97_0, 97_1, 97_2]"
98,Tom ist nicht tot; er ist quicklebendig.,Tom is not dead. He's alive and kicking.,"[Tom isn't dead; he'sde baked., Tom isn't dead...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[-1.0002425, -1.2548597, -0.84111214, -0.9629...","[98_0, 98_1, 98_2, 98_3, 98_4, 98_5, 98_6, 98_..."
99,Man konnte niemand die Schuld an dem Unfall ge...,Nobody was to blame for the accident.,"[You couldn't blame the accident on anyone., O...","[17, 1, 1, 1, 7, 1, 1, 1, 19, 1, 1, 3, 11, 1, ...","[[0.6651724, 0.6917383, -0.20953749, 0.5327471...","[99_0, 99_1, 99_2, 99_3, 99_4, 99_5, 99_6, 99_7]"


In [6]:
df_exploded = df.explode(["hypotheses", "hypotheses_idx"])

In [7]:
df_exploded

,source,target,hypotheses,references_count,utilities,hypotheses_idx
0,Ich habe dich nicht hereinkommen gehört.,I didn't hear you come in.,I didn't hear you come in.,"[9, 75, 3, 4, 1, 1, 2, 3, 1, 1]","[[0.9034017, 1.0596288, 0.80704784, 0.8429661,...",0_0
0,Ich habe dich nicht hereinkommen gehört.,I didn't hear you come in.,I didn't hear you coming in.,"[9, 75, 3, 4, 1, 1, 2, 3, 1, 1]","[[0.9034017, 1.0596288, 0.80704784, 0.8429661,...",0_1
1,Du wirst Tom küssen müssen.,You'll have to kiss Tom.,You're going to have to kiss Tom.,"[75, 21, 4]","[[1.1020814, 0.89811295, 0.96650434], [1.00817...",1_0
1,Du wirst Tom küssen müssen.,You'll have to kiss Tom.,You're going to need to kiss Tom.,"[75, 21, 4]","[[1.1020814, 0.89811295, 0.96650434], [1.00817...",1_1
1,Du wirst Tom küssen müssen.,You'll have to kiss Tom.,You'll need to kiss Tom.,"[75, 21, 4]","[[1.1020814, 0.89811295, 0.96650434], [1.00817...",1_2
...,...,...,...,...,...,...
99,Man konnte niemand die Schuld an dem Unfall ge...,Nobody was to blame for the accident.,There was no man but blaming the accident.,"[17, 1, 1, 1, 7, 1, 1, 1, 19, 1, 1, 3, 11, 1, ...","[[0.6651724, 0.6917383, -0.20953749, 0.5327471...",99_7
100,"Ich weiß, dass du es schaffen kannst.",I know you can make it.,I know you can do it.,"[66, 23, 5, 1, 3, 1, 1]","[[1.2360288, 0.98956245, 1.1368846, 0.68204087...",100_0
100,"Ich weiß, dass du es schaffen kannst.",I know you can make it.,I know that you can do it.,"[66, 23, 5, 1, 3, 1, 1]","[[1.2360288, 0.98956245, 1.1368846, 0.68204087...",100_1
100,"Ich weiß, dass du es schaffen kannst.",I know you can make it.,I know you can make it.,"[66, 23, 5, 1, 3, 1, 1]","[[1.2360288, 0.98956245, 1.1368846, 0.68204087...",100_2


In [8]:
from utilities.wrappers.CometWrapper import CometWrapper
from comet import download_model, load_from_checkpoint
# Next we load the comet model
model_path = download_model("wmt20-comet-da")
model = load_from_checkpoint(model_path)
model.to("cuda")
model.eval()
wrapped_model = CometWrapper(model)

C:\Users\gerso\anaconda3\envs\thesis\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wmt20-comet-da is already in cache.
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForS

In [9]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_exploded)

In [10]:
import torch

def to_embeddings(x):
    with torch.no_grad():
        embeddings = wrapped_model.to_embedding(x["hypotheses"]).cpu().numpy()
    
    return {"embedding": embeddings, **x}
    
new_dataset = dataset.map(to_embeddings , batched=True, batch_size=8)

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:05<00:00, 13.06ba/s]


In [11]:
new_df = new_dataset.to_pandas()

In [12]:
new_df["embedding"].iloc[0]

array([-0.14783731, -0.07063386,  0.17026192, ...,  0.2879284 ,
        0.19307423,  0.15560672], dtype=float32)

In [13]:
# Save it as a table
new_df = new_df[["hypotheses_idx", "embedding"]]

In [15]:
# Create a table out of it
table = {
    x["hypotheses_idx"]: torch.tensor(x["embedding"]) for _, x in new_df.iterrows()
    
}

In [16]:
table["0_0"].shape

torch.Size([1024])

In [17]:
### Next we need to create the reference table

ref_df = pd.read_parquet('../data/samples/ancestral_100_train_predictive_0_smoke_test.parquet')


In [19]:
ref_df["utilities"] = df["utilities"]

In [21]:
ref_df

,source,target,samples,sample_count,utilities
0,Ich habe dich nicht hereinkommen gehört.,I didn't hear you come in.,"[I didn't hear you coming in., I didn't hear y...","[9, 75, 3, 4, 1, 1, 2, 3, 1, 1]","[[0.9034017, 1.0596288, 0.80704784, 0.8429661,..."
1,Du wirst Tom küssen müssen.,You'll have to kiss Tom.,"[You're going to have to kiss Tom., You'll hav...","[75, 21, 4]","[[1.1020814, 0.89811295, 0.96650434], [1.00817..."
2,"Je mehr der Teufel hat, umso mehr will er.","The more the devil has, the more he wants.","[The more the devil has, the more he wants., T...","[67, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 4, 2, 1,...","[[1.0521425, 0.79392344, 0.3597772, -0.1302790..."
3,Mich hat das auch nicht überrascht.,"That didn't surprise me, either.","[I wasn't surprised at it either., I was not s...","[2, 1, 1, 4, 1, 1, 4, 1, 1, 1, 1, 1, 1, 6, 5, ...","[[1.1895545, 0.8660941, 0.94124323, 0.941016, ..."
4,Jedes Problem kann gelöst werden.,Every problem can be solved.,"[▁erzählte can be solved., Every problem can b...","[1, 30, 1, 24, 5, 7, 1, 1, 1, 3, 2, 1, 1, 2, 1...","[[0.6497966, 0.91674376, 1.0365593, 0.94450873..."
...,...,...,...,...,...
96,Wie lang ist es?,How long is it?,"[How long is it?, What's the length of it?, Fo...","[96, 1, 1, 1, 1]","[[1.1942372, 0.37009794, 0.82760555, -0.202470..."
97,Wir haben kein Bier mehr.,We're out of beer.,"[We've run out of beer., We have no more beer....","[50, 17, 3, 5, 17, 2, 2, 1, 1, 1, 1]","[[1.1632272, -0.17549993, -0.18429604, -0.3144..."
98,Tom ist nicht tot; er ist quicklebendig.,Tom is not dead. He's alive and kicking.,"[Tom is not dead. He is squirrel where he is.,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[-1.0002425, -1.2548597, -0.84111214, -0.9629..."
99,Man konnte niemand die Schuld an dem Unfall ge...,Nobody was to blame for the accident.,"[There was no one to blame for the accident., ...","[17, 1, 1, 1, 7, 1, 1, 1, 19, 1, 1, 3, 11, 1, ...","[[0.6651724, 0.6917383, -0.20953749, 0.5327471..."


In [22]:
ref_dataset = Dataset.from_pandas(ref_df)

In [37]:
def to_embeddings(x):
    with torch.no_grad():
        embeddings = wrapped_model.to_embedding(x["samples"]).cpu().numpy()
    
    return {"reference_embedding": embeddings, **x}
    
new_ref_dataset = ref_dataset.map(to_embeddings , batched=False)

100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [00:11<00:00,  8.69ex/s]


In [44]:
def to_source_embeddings(x):
    with torch.no_grad():
        embeddings = wrapped_model.to_embedding(x["source"]).cpu().numpy()
    
    return {"source_embedding": embeddings, **x}
new_ref_dataset = new_ref_dataset.map(to_source_embeddings , batched=True, batch_size=64)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.02s/ba]


In [45]:
new_ref_df = new_ref_dataset.to_pandas()

In [47]:
new_ref_df

,source,target,samples,sample_count,utilities,reference_embedding,source_embedding
0,Ich habe dich nicht hereinkommen gehört.,I didn't hear you come in.,"[I didn't hear you coming in., I didn't hear y...","[9, 75, 3, 4, 1, 1, 2, 3, 1, 1]","[[0.9034017, 1.0596288, 0.80704784, 0.8429661,...","[[-0.16658491, -0.046446856, 0.16569023, -0.09...","[-0.1885685, 0.16600133, 0.0124715185, -0.2320..."
1,Du wirst Tom küssen müssen.,You'll have to kiss Tom.,"[You're going to have to kiss Tom., You'll hav...","[75, 21, 4]","[[1.1020814, 0.89811295, 0.96650434], [1.00817...","[[-0.062416084, 0.08641304, 0.07719726, 0.0398...","[-0.3574337, 0.063110754, 0.2437705, -0.008016..."
2,"Je mehr der Teufel hat, umso mehr will er.","The more the devil has, the more he wants.","[The more the devil has, the more he wants., T...","[67, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 4, 2, 1,...","[[1.0521425, 0.79392344, 0.3597772, -0.1302790...","[[0.076547615, -0.17074764, -0.14683795, -0.41...","[-0.1427253, -0.14765261, 0.12876321, -0.41239..."
3,Mich hat das auch nicht überrascht.,"That didn't surprise me, either.","[I wasn't surprised at it either., I was not s...","[2, 1, 1, 4, 1, 1, 4, 1, 1, 1, 1, 1, 1, 6, 5, ...","[[1.1895545, 0.8660941, 0.94124323, 0.941016, ...","[[-0.14765848, -0.107647784, 0.057100426, -0.0...","[-0.13490665, -0.038284812, 0.07787761, -0.056..."
4,Jedes Problem kann gelöst werden.,Every problem can be solved.,"[▁erzählte can be solved., Every problem can b...","[1, 30, 1, 24, 5, 7, 1, 1, 1, 3, 2, 1, 1, 2, 1...","[[0.6497966, 0.91674376, 1.0365593, 0.94450873...","[[-0.045194414, -0.09846241, 0.041309938, -0.2...","[0.048856378, 0.02163599, 0.014341354, -0.1150..."
...,...,...,...,...,...,...,...
96,Wie lang ist es?,How long is it?,"[How long is it?, What's the length of it?, Fo...","[96, 1, 1, 1, 1]","[[1.1942372, 0.37009794, 0.82760555, -0.202470...","[[0.19093609, -0.25146833, 0.3576749, -0.17018...","[0.2368764, -0.23479639, 0.13985582, -0.246906..."
97,Wir haben kein Bier mehr.,We're out of beer.,"[We've run out of beer., We have no more beer....","[50, 17, 3, 5, 17, 2, 2, 1, 1, 1, 1]","[[1.1632272, -0.17549993, -0.18429604, -0.3144...","[[-0.048731085, -0.007297343, 0.2508326, -0.03...","[-0.14855956, 0.014186058, 0.37380266, 0.00179..."
98,Tom ist nicht tot; er ist quicklebendig.,Tom is not dead. He's alive and kicking.,"[Tom is not dead. He is squirrel where he is.,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[-1.0002425, -1.2548597, -0.84111214, -0.9629...","[[-0.1592762, -0.07902474, -0.18203558, -0.003...","[0.030717136, 0.009531055, -0.002753221, -0.13..."
99,Man konnte niemand die Schuld an dem Unfall ge...,Nobody was to blame for the accident.,"[There was no one to blame for the accident., ...","[17, 1, 1, 1, 7, 1, 1, 1, 19, 1, 1, 3, 11, 1, ...","[[0.6651724, 0.6917383, -0.20953749, 0.5327471...","[[0.022380723, -0.21277903, 0.13047011, -0.131...","[-0.22929758, -0.060340185, 0.18796425, -0.256..."


In [48]:
new_ref_df.columns

Index(['source', 'target', 'samples', 'sample_count', 'utilities',
       'reference_embedding', 'source_embedding'],
      dtype='object')

In [50]:
# for _, x in new_ref_df.iterrows():
#     print(x)
ref_table = {
    x.name: {
        "source_embedding": x["source_embedding"],
        "reference_embedding": x["reference_embedding"],
        "utilities": x["utilities"]
    } for _, x in new_ref_df.iterrows()
}


In [51]:
ref_table

{0: {'source_embedding': array([-0.1885685 ,  0.16600133,  0.01247152, ...,  0.10462446,
         -0.00420072,  0.05064526], dtype=float32),
  'reference_embedding': array([array([-0.16658491, -0.04644686,  0.16569023, ...,  0.30325407,
                 0.19095343,  0.05838242], dtype=float32)               ,
         array([-0.14842065, -0.07091627,  0.17087328, ...,  0.28854948,
                 0.19357881,  0.15608364], dtype=float32)               ,
         array([-0.15202312, -0.08534526,  0.11832917, ...,  0.30928865,
                 0.18998687,  0.1055574 ], dtype=float32)               ,
         array([-0.07087111, -0.07381896,  0.11993284, ...,  0.26931003,
                 0.22873825,  0.19050598], dtype=float32)               ,
         array([-0.0638062 , -0.06083072,  0.17014077, ...,  0.21359456,
                 0.13965803,  0.18534394], dtype=float32)               ,
         array([-0.16530575, -0.10121386,  0.2804619 , ...,  0.35590503,
                 0.15745622,